In [27]:
from Bio.Seq import Seq
from Bio.Align import MultipleSeqAlignment
from Bio import AlignIO, SeqIO
import panel as pn
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource, Plot, Grid, Range1d
from bokeh.models.glyphs import Text, Rect
from bokeh.layouts import gridplot
from bokeh.io import export_svg, output_notebook, install_notebook_hook, notebook
import numpy as np

In [2]:
clrs =  {"A":"red","T":"green","G":"orange","C":"blue","-":"white", " ":"white","D":"red","E":"lightblue", "F":"tan", "H":"peru", "I":"teal", "K":"olive", "L":"darkseagreen",
             "M":"coral", "N":"orange", "P":"pink", "Q":"violet", "R":"mediumpurple", "S":"slateblue", "T":"salmon", "V":"lightcoral", "W":"indigo", "Y":"slateblue"}

In [36]:
def get_colors(ref, seqs, wins, longest):
    """make colors for bases in sequence"""
    text = [i for s in list(seqs) for i in s]
    clrs = ["steelblue", "palegreen", "coral", "cadetblue", "palevioletred", "mediumaquamarine", "plum", "salmon"]
    colors = ["white" for i in text]
    for idx, pair in enumerate(zip(seqs, wins)):
        clr = 0
        for window in pair[1]:
            start = window[0]
            end = window[1]
            for i in range(start, end+1):
                colors[idx * longest + i] = clrs[clr%len(clrs)]
            clr += 1
    return colors

def merge_windows(wins):
    a = []
    for alignment in wins:
        k = [alignment[0]]
        for win in alignment[1:]:
            if k[len(k)-1][1] >= (win[0]-1):
                k[len(k)-1] = (k[len(k)-1][0],win[1])
            else:
                k.append(win)
        a.append(k)
    return a

def read_windows(path):
    lines = []
    with open(path, "r") as f:
        lines = f.readlines()
    ref = lines[0].split(" ")[1]
    longest = len(ref)
    # for line in lines[2:]:
    #     longest = max(longest, len(line.split(" ")[1]))
    seqs = []
    windows = []
    for line in lines[2:]:
        seq = line.split(" ")[1]
        a = list(map(int, line.split(" ")[3:]))
        wins = list(zip(a[::2], a[1::2]))
        seqs.append(ref)
        windows.append(wins)
    return (ref, seqs, windows, longest)
    

In [37]:
def plot_windows(path):
    ref, seqs, wins, longest = read_windows(path)
    wins = merge_windows(wins)
    text = [i for s in list(seqs) for i in s]
    colors = get_colors(ref, seqs, wins, longest)
    cols = longest
    rows = len(wins)
    x = np.arange(1, cols+1)
    y = np.arange(0, rows, 1)
    xx, yy = np.meshgrid(x, y)
    gx = xx.ravel()
    gy = yy.flatten()
    recty = gy + 0.5
    h = 1 / cols
    source = ColumnDataSource(dict(x=gx, y=gy, recty=recty,text=text, colors=colors))
    plot_height = len(seqs)*10+50
    plot_width = longest * 5
    x_range = Range1d(0,cols+1, bounds='auto')
    if cols>100:
        viewlen=100
    else:
        viewlen=cols
    #view_range is for the close up view
    view_range = (0,viewlen)

    rects = Rect(x="x", y="recty",  width=1, height=1, fill_color="colors", line_color=None, fill_alpha=0.8)
    glyph = Text(x="x", y="y", text="text", text_align='center',text_color="black", text_font_size="4pt")
    
    p = figure(title=None, plot_width=plot_width, plot_height=plot_height,
               x_range=x_range, y_range=(0,rows),
               min_border=0, toolbar_location='below')
    p.add_glyph(source, rects)
    p.add_glyph(source, glyph)    
    p = gridplot([[p]])
    return p
    

In [42]:
p = plot_windows("./../out/uniprot_sprot.40.multi-step/safety/A1K9S9.out")
show(p)

